<h2>Dados de voos com origem ou destino no Brasil de janeiro a março/24</h2>

Dados da ANAC

In [28]:
import os
import pandas as pd
import numpy as np
import datetime

pasta = r"C:\Users\bserpellone\Desktop\Python\Dataset\anac"

registros_abr = "VRA_2024_03.csv"
dAeroportos = "glossario_de_aerodromo.xls"
dViacoes = "glossario_de_empresas_aereas.xls"

df_concat = pd.DataFrame(columns=['Sigla ICAO Empresa Aérea', 'Número Voo', 'Código DI',
       'Código Tipo Linha', 'Sigla ICAO Aeroporto Origem',
       'Partida Prevista', 'Partida Real', 'Sigla ICAO Aeroporto Destino',
       'Chegada Prevista', 'Chegada Real', 'Situação Voo'])
for registros in os.listdir(pasta):
    if registros.lower().startswith('vra'):
        df = pd.read_csv(os.path.join(pasta,registros),delimiter=';')
        df_concat = pd.concat(objs=[df_concat,df])

df_aeros = pd.read_excel(os.path.join(pasta,dAeroportos),header=3)
df_aeros.dropna(axis=1,how='all',inplace=True)

df_viacoes = pd.read_excel(os.path.join(pasta,dViacoes),header=3)
df_viacoes.dropna(axis=1,how='all',inplace=True)

df_merged = df_concat.merge(df_viacoes,how='left',left_on='Sigla ICAO Empresa Aérea',right_on='Sigla OACI')
df_merged = df_merged.merge(df_aeros,left_on='Sigla ICAO Aeroporto Origem',right_on='Sigla OACI',how='left',suffixes=('_ori','_dest'))
df_merged = df_merged.merge(df_aeros,left_on='Sigla ICAO Aeroporto Destino',right_on='Sigla OACI',how='left',suffixes=('_Origem','_Destino'))

<h4>Quantas linhas/registros de voos tem na base?</h4>

In [29]:
df_selecao = df_merged[['Chegada Real','Situação Voo','Nome Empresas','Nacional ou Estrangeira','Partida Prevista','Partida Real','Cidade_Origem','UF_Origem', 'País_Origem','Continente_Origem','Cidade_Destino', 'UF_Destino', 'País_Destino','Continente_Destino']]
df_selecao.shape[0]

326725

Primeiras linhas da tabela:

In [30]:
df_selecao.head(5)

,Chegada Real,Situação Voo,Nome Empresas,Nacional ou Estrangeira,Partida Prevista,Partida Real,Cidade_Origem,UF_Origem,País_Origem,Continente_Origem,Cidade_Destino,UF_Destino,País_Destino,Continente_Destino
0,02/01/2024 08:19,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,01/01/2024 23:55,01/01/2024 23:47,RIO DE JANEIRO,RJ,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE
1,01/01/2024 09:35,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,01/01/2024 23:55,01/01/2024 01:29,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,RIO DE JANEIRO,RJ,BRASIL,AMERICA DO SUL
2,01/01/2024 08:45,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,01/01/2024 00:55,01/01/2024 00:46,GUARULHOS,SP,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE
3,02/01/2024 07:47,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,01/01/2024 21:20,01/01/2024 23:17,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,GUARULHOS,SP,BRASIL,AMERICA DO SUL
4,02/01/2024 06:13,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,01/01/2024 20:50,01/01/2024 21:51,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,GUARULHOS,SP,BRASIL,AMERICA DO SUL


<h4>Qual a proporção de voos domésticos e estrangeiros realizados?</h4>

In [66]:
df_selecao['Nacional ou Estrangeira'][df_selecao['Situação Voo']=='REALIZADO'].value_counts(normalize=True).round(2)

Nacional ou Estrangeira
BRASILEIRA       0.9
ESTRANGEIRA      0.1
NÃO INFORMADO    0.0
Name: proportion, dtype: float64

In [67]:
df_selecao['Cidade_Origem'][(df_selecao['Nacional ou Estrangeira']=='BRASILEIRA') & (df_selecao['Situação Voo']=='REALIZADO')].value_counts(normalize=True).round(2).head(5)

Cidade_Origem
GUARULHOS         0.13
SÃO PAULO         0.12
RIO DE JANEIRO    0.08
CAMPINAS          0.07
BRASÍLIA          0.06
Name: proportion, dtype: float64

<h4>Quais as principais cidades origem dos voos?</h4>

In [69]:
df_selecao['Cidade_Origem'][df_selecao['Situação Voo']=='REALIZADO'].value_counts(normalize=True).round(2).head(5)

Cidade_Origem
GUARULHOS         0.14
SÃO PAULO         0.10
RIO DE JANEIRO    0.08
CAMPINAS          0.06
BRASÍLIA          0.06
Name: proportion, dtype: float64

<h4>Quais as principais cidades destino dos voos?</h4>

In [70]:
df_selecao['Cidade_Destino'].value_counts(normalize=True).round(2).head(5)

Cidade_Destino
GUARULHOS         0.14
SÃO PAULO         0.10
RIO DE JANEIRO    0.08
CAMPINAS          0.06
BRASÍLIA          0.06
Name: proportion, dtype: float64

<h4>Quais foram as 5 horas mais frequentes de voos pousando no Brasil (seja doméstico ou vindo do exterior)?</h4>

In [71]:
df_selecao['Chegada Real'].str[11:13][(df_selecao['UF_Destino'].notna()) & (df_selecao['Situação Voo']=='REALIZADO')].value_counts(normalize=True).round(3).head(5)

Chegada Real
16    0.062
11    0.061
19    0.058
10    0.056
07    0.055
Name: proportion, dtype: float64

Quais foram as 5 horas mais frequentes de voos decolando no Brasil (seja doméstico ou indo para o exterior)?

In [72]:
df_selecao['Chegada Real'].str[11:13][(df_selecao['UF_Origem'].notna()) & (df_selecao['Situação Voo']=='REALIZADO')].value_counts(normalize=True).round(3).head(5)

Chegada Real
11    0.064
16    0.058
10    0.058
13    0.056
07    0.056
Name: proportion, dtype: float64

<h4>Qual é a proporção de destinos por continente em voos internacionais que saíram do Brasil?</h4>

In [65]:
df_selecao['Continente_Destino'][(df_selecao['Nacional ou Estrangeira']=='ESTRANGEIRA') & (df_selecao['Situação Voo']=='REALIZADO')].value_counts(normalize=True).round(2)

Continente_Destino
AMERICA DO SUL      0.71
EUROPA              0.13
AMERICA DO NORTE    0.09
AMERICA CENTRAL     0.05
ASIA                0.02
AFRICA              0.01
Name: proportion, dtype: float64

<h4> Quais os principais destinos internacionais (países)?</h4>

In [73]:
df_selecao['País_Destino'][(df_selecao['País_Destino']!='BRASIL') & (df_selecao['Situação Voo']=='REALIZADO')].value_counts(normalize=True).round(2).head(5)

País_Destino
ARGENTINA                    0.20
ESTADOS UNIDOS DA AMERICA    0.16
CHILE                        0.13
PORTUGAL                     0.07
PANAMA                       0.05
Name: proportion, dtype: float64

<h4>Quais os principais destinos internacionais (cidades)?</h4>

In [88]:
df_selecao.loc[:,'Cidade_Destino'] = df_selecao['Cidade_Destino'].replace("BUENOS AIRES/AEROPARQUE","BUENOS AIRES")
df_selecao['Cidade_Destino'][(df_selecao['País_Destino']!='BRASIL') & (df_selecao['Situação Voo']=='REALIZADO')].value_counts(normalize=True).round(2).head(7)

Cidade_Destino
BUENOS AIRES         0.19
SANTIAGO DO CHILE    0.13
LISBOA               0.07
MIAMI                0.06
PANAMA               0.05
BOGOTA               0.05
MONTEVIDEU           0.04
Name: proportion, dtype: float64

<h4>Como é a proporção de viagens por aviação?</h4>

In [75]:
df_selecao['Nome Empresas'][df_selecao['Situação Voo']=='REALIZADO'].value_counts(normalize=True).round(2).head(5)

Nome Empresas
AZUL                0.33
TAM                 0.29
GOL                 0.24
PASSAREDO           0.02
TAP AIR PORTUGAL    0.01
Name: proportion, dtype: float64

<h4>Como é a proporção de viagens por aviação estrangeira?</h4>

In [76]:
df_selecao['Nome Empresas'][(df_selecao['Nacional ou Estrangeira']=="ESTRANGEIRA") & (df_selecao['Situação Voo']=='REALIZADO')].value_counts(normalize=True).round(2).head(5)

Nome Empresas
TAP AIR PORTUGAL                       0.11
LAN CHILE                              0.10
AEROLINEAS ARGENTINAS                  0.09
COPA -COMPANIA PANAMENA DE AVIACION    0.09
AMERICAN AIRLINES INC                  0.05
Name: proportion, dtype: float64

<h4> Houve outras aviações fora Azul, Tam, Gol e Passaredo em voos nacionais?</h4>

In [77]:
df_selecao['Nome Empresas'][(df_selecao['Nacional ou Estrangeira']=="BRASILEIRA") & (df_selecao['Situação Voo']=='REALIZADO')].value_counts(normalize=True).round(2)

Nome Empresas
AZUL                 0.37
TAM                  0.33
GOL                  0.27
PASSAREDO            0.02
MAP LINHAS AEREAS    0.01
ABSA                 0.00
TOTAL                0.00
ABAETE               0.00
Name: proportion, dtype: float64

<h4>Como foi o status dos voos em relação a voos nacionais e internacionais?</h4>

In [43]:
df_selecao.groupby('Nacional ou Estrangeira')['Situação Voo'].value_counts(normalize=True).round(2)

Nacional ou Estrangeira  Situação Voo 
BRASILEIRA               REALIZADO        0.97
                         CANCELADO        0.03
ESTRANGEIRA              REALIZADO        0.84
                         CANCELADO        0.16
                         NÃO INFORMADO    0.00
NÃO INFORMADO            REALIZADO        0.76
                         CANCELADO        0.24
Name: proportion, dtype: float64

In [45]:
df_selecao['Situação Voo'][df_selecao['Nacional ou Estrangeira']=='ESTRANGEIRA'].value_counts()

Situação Voo
REALIZADO        28992
CANCELADO         5595
NÃO INFORMADO        2
Name: count, dtype: int64

<h4>A proporção de voos estrangeiros cancelados é cerca de 5x comparado aos voos domésticos. Como fica a proporção comparando voos que partem do Brasil para o exterior com os que partem do exterior para o Brasil?</h4>

In [48]:
df_selecao['Situação Voo'][(df_selecao['Nacional ou Estrangeira']=='ESTRANGEIRA') & (df_selecao['UF_Origem'].notna())].value_counts(normalize=True).round(2)

Situação Voo
REALIZADO        0.91
CANCELADO        0.09
NÃO INFORMADO    0.00
Name: proportion, dtype: float64

In [49]:
df_selecao['Situação Voo'][(df_selecao['Nacional ou Estrangeira']=='ESTRANGEIRA') & (df_selecao['UF_Origem'].isna())].value_counts(normalize=True).round(2)

Situação Voo
REALIZADO        0.78
CANCELADO        0.22
NÃO INFORMADO    0.00
Name: proportion, dtype: float64

<h4>A maior % de voos cancelados partem do exterior com destino ao Brasil. Quais as principais origens desses voos?</h4>

In [63]:
df_selecao['País_Origem'][(df_selecao['Nacional ou Estrangeira']=='ESTRANGEIRA') & (df_selecao['UF_Origem'].isna()) & (df_selecao['Situação Voo']=='CANCELADO')].value_counts(normalize=True).round(2).head(5)

País_Origem
ARGENTINA                    0.23
ESTADOS UNIDOS DA AMERICA    0.12
ESPANHA                      0.09
QATAR                        0.08
PANAMA                       0.07
Name: proportion, dtype: float64

<h4>Qual a proporção de voos atrasados?</h4>

Valores da coluna situação voo:

In [54]:
df_selecao['Situação Voo'].drop_duplicates()

0             REALIZADO
30            CANCELADO
195333    NÃO INFORMADO
Name: Situação Voo, dtype: object

In [55]:
df_selecao.head(3)

,Chegada Real,Situação Voo,Nome Empresas,Nacional ou Estrangeira,Partida Prevista,Partida Real,Cidade_Origem,UF_Origem,País_Origem,Continente_Origem,Cidade_Destino,UF_Destino,País_Destino,Continente_Destino
0,02/01/2024 08:19,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,01/01/2024 23:55,01/01/2024 23:47,RIO DE JANEIRO,RJ,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE
1,01/01/2024 09:35,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,01/01/2024 23:55,01/01/2024 01:29,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,RIO DE JANEIRO,RJ,BRASIL,AMERICA DO SUL
2,01/01/2024 08:45,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,01/01/2024 00:55,01/01/2024 00:46,GUARULHOS,SP,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE


<h4>Como não tem coluna indicado se o voo atrasou, vamos calcular operando as colunas partida real x partida prevista:</h4>

In [56]:
df_status = df_selecao.copy()

<h4>Nessa análise quero manter apenas os voos realizados</h4>

In [57]:
df_status = df_status[df_status['Situação Voo'] == 'REALIZADO']

<h4>Excluindo linhas sem informação de partida real ou partida prevista:</h4>

In [58]:
df_status[['Partida Real','Partida Prevista']].dropna(how='any')

,Partida Real,Partida Prevista
0,01/01/2024 23:47,01/01/2024 23:55
1,01/01/2024 01:29,01/01/2024 23:55
2,01/01/2024 00:46,01/01/2024 00:55
3,01/01/2024 23:17,01/01/2024 21:20
4,01/01/2024 21:51,01/01/2024 20:50
...,...,...
326716,11/04/2024 16:25,11/04/2024 16:40
326719,18/04/2024 15:20,18/04/2024 13:30
326720,18/04/2024 18:37,18/04/2024 16:40
326721,25/04/2024 13:23,25/04/2024 13:30


In [59]:
df_status[['Partida Real','Partida Prevista']].head(5)

,Partida Real,Partida Prevista
0,01/01/2024 23:47,01/01/2024 23:55
1,01/01/2024 01:29,01/01/2024 23:55
2,01/01/2024 00:46,01/01/2024 00:55
3,01/01/2024 23:17,01/01/2024 21:20
4,01/01/2024 21:51,01/01/2024 20:50


<h4>Abaixo calculamos a diferença, em segundos, entre as 2 colunas</h4>

In [60]:
df_status.loc[:,'Diferença_segundos'] = (pd.to_datetime(df_status['Partida Real'], errors='coerce') - pd.to_datetime(df_status['Partida Prevista'], errors='coerce')).dt.total_seconds()
df_status[['Partida Real','Partida Prevista','Diferença_segundos']].head(5)

,Partida Real,Partida Prevista,Diferença_segundos
0,01/01/2024 23:47,01/01/2024 23:55,-480.0
1,01/01/2024 01:29,01/01/2024 23:55,-80760.0
2,01/01/2024 00:46,01/01/2024 00:55,-540.0
3,01/01/2024 23:17,01/01/2024 21:20,7020.0
4,01/01/2024 21:51,01/01/2024 20:50,3660.0


<h4>Observamos que Diferença_segundos < 0 indicam voos adiantados e Diferença_segundos > 0, atrasados. Assim, vamos categorizá-los:</h4>

In [61]:
# Pontual = diferença da hora prevista para realizada de 5 minutos ou menos, seja adiantada ou atrasada
df_status.loc[df_status['Diferença_segundos'].abs() <= 300, 'Status_voo'] = 'Pontual'

# Atraso de 5 a 20 min:
df_status.loc[(df_status['Diferença_segundos'].abs() > 300) & (df_status['Diferença_segundos'].abs() <= 1200), 'Status_voo'] = 'De 5 a 20min atrasado/adiantado'

# Atraso de mais de 20min
df_status.loc[df_status['Diferença_segundos'].abs() > 1200, 'Status_voo'] = '20min+ atrasado/adiantado'

In [62]:
df_status['Status_voo'].value_counts(normalize=True).round(2)

Status_voo
De 5 a 20min atrasado/adiantado    0.44
Pontual                            0.40
20min+ atrasado/adiantado          0.16
Name: proportion, dtype: float64